- using https://database.itreetools.org/#/viableLocations for locations in California that has data in the system
- collect html with https://ourtrees.itreetools.org/#/report? http get method web link
- HTMLs are collect within Los Angeles County only.
- Some data was wrongly input to database and can not find the right location.  It was deleted in LocationsList_2025-05-25.csv

In [ ]:
import pandas as pd
import requests
import os
from datetime import datetime
from lxml import html
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time

if not os.path.exists('html_files'):
    os.mkdir('html_files')

locationList = pd.read_csv("data/LocationsList_2025-05-25.csv")
zipcode = pd.read_csv("data/zip_code_data.csv")
#print(zipcode.columns)
print(locationList.columns)
unique_values = locationList['Place'].unique()
print(len(unique_values))
pasadena = locationList[locationList['Place'] == 'Pasadena']
print(pasadena.iloc[0])
cityInCounty = locationList[locationList['County'] == 'Los Angeles']
cityInCounty = cityInCounty.drop_duplicates(subset='Place', keep='first')
print(cityInCounty.head(5))

filename = f'html_files/CA_pasadena.html'
Longitude = pasadena.iloc[0]['Longitude']
Latitude = pasadena.iloc[0]['Latitude']
url = f'https://ourtrees.itreetools.org/#/report?longitude={Longitude}&latitude={Latitude}&tab=benefits'
print(url)
# Make a GET request to the URL and save the response content to a file
options = Options()
options.add_argument("headless")  # Don't show browser window


accuTime = 0
print(len(cityInCounty))
for index, row in cityInCounty.iterrows():
    filename = "html_files/CA_" + row['Place'] + ".html"

    genHTML = True
    if os.path.exists(filename) :
        if (os.path.getsize(filename)/ 1024) > 50:
            genHTML = False
    
    if genHTML:
        print(filename)
        
        driver = webdriver.Chrome(options=options)
        url = "https://ourtrees.itreetools.org/#/report?longitude="+ str(row['Longitude']) + "&latitude=" + str(row['Latitude']) + "&tab=benefits"
        
        print(url)
        driver.get(url)
        # wait for the page to finish rendering
        time.sleep(30)  # or use WebDriverWait with specific DOM condition
        html = driver.page_source
        accuTime = 0
        while 'Estimating' in html:
            time.sleep(3)
            html = driver.page_source
            accuTime = accuTime + 3
            if accuTime > 480:
                break
        
        with open(filename, 'w') as f:
            f.write(html)
        #print(html)
        driver.quit()
        time.sleep(3)



Index(['Nation', 'State', 'County', 'Place', 'Latitude', 'Longitude',
       'Elevation (meters)'],
      dtype='object')
1682
Nation                United States of America
State                               California
County                             Los Angeles
Place                                 Pasadena
Latitude                             34.176609
Longitude                          -118.172614
Elevation (meters)                  280.573425
Name: 604, dtype: object
                       Nation       State       County                Place  \
506  United States of America  California  Los Angeles                Acton   
507  United States of America  California  Los Angeles         Agoura Hills   
508  United States of America  California  Los Angeles  Agoura Hills-Malibu   
509  United States of America  California  Los Angeles           Agua Dulce   
510  United States of America  California  Los Angeles             Alhambra   

      Latitude   Longitude  Elevation (meter

In [ ]:
from pyquery import PyQuery
import re

def getDictItem(pairs, keys):
    if len(pairs) > 1:
        col = items(pairs[0]).text().replace(":", "")
        colstr = col.split(" ")
        if len(colstr) > 3:
            col = " ".join([colstr[0], colstr[1], colstr[2]])
        
        value = items(pairs[1]).text()
        if len(value.split(" ")) > 1:
            col = col + "(" + value.split(" ")[1] + ")"
            value = value.split(" ")[0]
        if "$" in value:
            col = col + "($)"

        value = re.sub(r"[$,]", "", value)

        if col in keys:
            col = col + "-2"
        return {col: value}
    return {}
    
with open(filename, 'r',encoding='ISO-8859-1') as f:
    html_str = f.read()
    doc = PyQuery(html_str)
    print(filename)
    row = {"City": re.split(r"[./]", filename)[1]}
    # parse benefits page
    labels = [ "City", "percent canopy", "Total Land(acres)", "percent impervious", "impervious(acres)", "Total i-Tree benefits($)", "Carbon Dioxide Uptake($)", "Carbon Sequestered(tn)",
              "CO2 Equivalent1(tn)", "Storm Water Mitigation($)", "Runoff Avoided(MG/yr)", "Rainfall Intercepted(MG/yr)", "Air Pollution Removal($)",
              "Carbon Monoxide(lb/yr)", "Ozone(lb/yr)", "Nitrogen Dioxide(lb/yr)", "Sulfur Dioxide(lb/yr)", "PM2.5(lb/yr)", "Carbon Dioxide Uptake-2($)", "Carbon Storage(tn)", "CO2 Equivalent1-2(tn)"]
    
    benefits = doc("#benefits")
    items = benefits(".justify-content-between")
    for i, item in enumerate(items.items(), 1):
        strong = item.find("strong")
        row = row | getDictItem(strong, row.keys())
        p = item.find("p")
        row = row | getDictItem(p, row.keys())

    topitem = benefits(".my-0")
    toplist1 = items(topitem[0]).text().split(" ")
    toplist2 = items(topitem[1]).text().split(" ")
    print(toplist1)
    print(toplist2)
    row["percent canopy"] = re.sub(r"[%]", "", toplist1[0])
    row["Total Land(acres)"] = re.sub(r",", "", toplist1[4])
    row["percent impervious"] = re.sub(r"[%]", "", toplist2[0])
    row["impervious(acres)"] = re.sub(r",", "", toplist2[4])

    for key, value in row.items():
        print(f"{key}: {value}")
    print(len(row.keys()))

    Community = doc("#Community")
    
    


html_files/CA_pasadena.html
['13.19%', 'tree', 'canopy', 'on', '1,939', 'acres']
['38.08%', 'impervious', 'surfaces', 'over', '5,598', 'acres']
City: CA_pasadena
Total i-Tree benefits($): 3039197
Carbon Dioxide Uptake($): 1498517
Carbon Sequestered(tn): 3463
CO2 Equivalent1(tn): 12698
Storm Water Mitigation($): 256108
Runoff Avoided(MG/yr): 29
Rainfall Intercepted(MG/yr): 76
Air Pollution Removal($): 1284572
Carbon Monoxide(lb/yr): 4563
Ozone(lb/yr): 162164
Nitrogen Dioxide(lb/yr): 29180
Sulfur Dioxide(lb/yr): 5879
PM2.5(lb/yr): 2450
Carbon Dioxide Uptake($)-2: 29785452
Carbon Storage(tn): 68832
CO2 Equivalent1(tn)-2: 252384
percent canopy: 13.19
Total Land(acres): 1939
percent impervious: 38.08
impervious(acres): 5598
21


In [ ]:
import geopandas as gpd
import json
import os

fileMerge = [
    "Street_ROW_Trees.geojson", 
    "merged_pasadena_2048_0.6m_baseline.geojson",
    "deep_forest_init.geojson", 
    "pasadena_predictions_4326_10cm.geojson",
    "merged_pasadena_60cm_cv.geojson",
    "pasadena_merged_polygons_4326.geojson"
]

treePoints = ["training", "baseline", "deepforest", "deepforest10cm", "cv60cm", "deepforest10cmMerged"]
treeType = ["pt", "pt", "box", "box", "pt", "box"]

geojson_dir = "./greencityEc2/geojson/"  # Adjust if your files are stored elsewhere

blocks = gpd.read_file(os.path.join(geojson_dir, "2010_Census_Blocks.geojson"))

projected_crs = "EPSG:3857"

def process_model(file, model, tree_type, blocks):
    print(f"Processing model '{model}' from file '{file}' (treeType={tree_type}) ...")

    trees = gpd.read_file(os.path.join(geojson_dir, file))

    # Ensure CRS matches
    if trees.crs != blocks.crs:
        trees = trees.to_crs(blocks.crs)

    # Project both to meters
    blocks_proj = blocks.to_crs(projected_crs)
    trees_proj = trees.to_crs(projected_crs)

    blocks_proj["areaSqM"] = blocks_proj.geometry.area

    if tree_type == "pt":
        joined = gpd.sjoin(trees_proj, blocks_proj, how="inner", predicate="within")
    elif tree_type == "box":
        trees_proj["center"] = trees_proj.geometry.centroid
        rect_centers = trees_proj.copy()
        rect_centers.set_geometry("center", inplace=True)
        joined = gpd.sjoin(rect_centers, blocks_proj, how="inner", predicate="within")

    joined_latlon = joined.to_crs("EPSG:4326")

    tree_counts = joined.groupby("index_right").size().rename("treeCount")
    blocks_proj["treeCount"] = blocks_proj.index.map(tree_counts).fillna(0).astype(int)
    blocks_proj["density"] = (blocks_proj["treeCount"] / blocks_proj["areaSqM"]) * 10000

    density_dict = dict(zip(blocks_proj["OBJECTID"], blocks_proj["density"]))

    if tree_type == "pt":
        tree_coords = joined_latlon.groupby("index_right")["geometry"].apply(
            lambda pts: [pt.coords[0] for pt in pts if pt.geom_type == "Point"]
        )
    elif tree_type == "box":
        tree_coords = joined_latlon.groupby("index_right")["center"].apply(
            lambda pts: [pt.coords[0] for pt in pts]
        )

    geojson_dict = json.loads(trees.to_json())
    features = geojson_dict.get("features", [])

    print(f"Finished processing model '{model}'.")
    return {
        "density": density_dict,
        "features": features,
        "coords": tree_coords.to_dict() if tree_coords is not None else {}
    }

# Loop over all files and process
for index in range(len(fileMerge)):
    js_filename = f"{treePoints[index]}.js"
    if os.path.exists(js_filename):
        continue

    model_data = process_model(fileMerge[index], treePoints[index], treeType[index], blocks)

    js_lines = []

    # Optionally add processed geojson for map use
    # js_variable_name = "census_blocks_" + treePoints[index]
    # js_lines.append(f"window.{js_variable_name} = {json.dumps(model_data['features'])};")

    # Add tree GeoJSON
    trees = gpd.read_file(os.path.join(geojson_dir, fileMerge[index]))
    trees_json = json.loads(trees.to_json())
    js_lines.append(f"window.{treePoints[index]} = {json.dumps(trees_json)};")

    # Write JavaScript file
    full_js_content = "\n\n".join(js_lines)
    with open(js_filename, "w", encoding="utf-8") as f:
        f.write(full_js_content)


Processing model 'deepforest' from file 'deep_forest_init.geojson' (treeType=box) ...
Finished processing model 'deepforest'.
Processing model 'deepforest10cm' from file 'pasadena_predictions_4326_10cm.geojson' (treeType=box) ...
Finished processing model 'deepforest10cm'.
Processing model 'cv60cm' from file 'merged_pasadena_60cm_cv.json' (treeType=pt) ...


DataSourceError: ./greencityEc2/geojson/merged_pasadena_60cm_cv.json: No such file or directory